This Notebook launches a convergence study for the Bow Shock Problem as presented in the paper "An Extended Discontinuous Galerkin Method for High-order shock treatment" (2023) 
 The calculations can be very long. 
 In order to use a HPC cluster ``..\.BoSSS\etc\BatchProcessorConfig.json`` must be modified for the execution queue

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


get date for naming the workflow an Database

In [2]:
var date = DateTime.Now.ToString("ddMMyyyy");
date

25082023

init workflow


In [3]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy2_" + date );
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_ConvStudy2_25082023'.
Default Execution queue is chosen for the database.
Creating database '\\dc1\userspace\sebastian\cluster\XESF_BowShock_ConvStudy2_25082023'.


{ Session Count = 0; Grid Count = 0; Path = \\dc1\userspace\sebastian\cluster\XESF_BowShock_ConvStudy2_25082023 }

Get Default Queue

In [4]:
BoSSSshell.ExecutionQueues

index,type,RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths,BatchInstructionDir
0,BoSSS.Application.BoSSSpad.MsHPC2012Client,win\amd64,\\dc1\userspace\sebastian\cluster\binaries,True,Default,dotnet,FDY\sebastian,DC2,"[ hpccluster, hpccluster2, hpcluster3, hpccluster4 ]",Normal,True,List \\dc1\userspace\sebastian\cluster,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,win\amd64,\\dc1\userspace\sebastian\cluster\binaries,True,Default,dotnet,FDY\sebastian,DC2,[ hpccluster4 ],Normal,True,List \\dc1\userspace\sebastian\cluster,
2,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,<null>,C:\Users\sebastian\AppData\Local\BoSSS-LocalJobs,False,<null>,dotnet,,,,,,List C:\Users\sebastian C:\,<null>


default database path

In [5]:
BoSSSshell.WorkflowMgm.DefaultDatabase.Path

\\dc1\userspace\sebastian\cluster\XESF_BowShock_ConvStudy2_25082023

list of available queues

In [6]:
BoSSSshell.GetDefaultQueue()

RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
win\amd64,\\dc1\userspace\sebastian\cluster\binaries,True,Default,dotnet,FDY\sebastian,DC2,"[ hpccluster, hpccluster2, hpcluster3, hpccluster4 ]",Normal,True,List<AllowedDatabasesPair> \\dc1\userspace\sebastian\cluster


Here we prescribe all combinations we want to study

In [7]:
using XESF.Fluxes;
using ApplicationWithIDT;

int[] DGdegrees = new int[] {3};
int[] Grid_Res = new int[] {1,2,4,8};
double[] Aggs = new double[] {0.4};
var iFluxes = new int[] {0,1}; //{ ConvectiveInterfaceFluxes.GodunovInterface,ConvectiveInterfaceFluxes.RoeInterface, ConvectiveInterfaceFluxes.CentralFluxInterface, ConvectiveInterfaceFluxes.OptimizedHLLCInterface };
var iProbs= new int[] {0}; //{ OptProblemType.FullEnRes, OptProblemType.EnResOnlyNearBand, OptProblemType.RankineHugoniot};
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;  


get data we need to initialize the initial guess for the shock LevelSet

In [8]:
var bowShockPoints =new Tuple<byte[],string>( System.IO.File.ReadAllBytes("BowShockPoints.txt"),"BowShockPoints.txt");
var LevelSetDB =new Tuple<byte[],string>(System.IO.File.ReadAllBytes("bosss_db_levelSets.zip"),"bosss_db_levelSets.zip");

run the study

In [9]:
foreach(double iProb in iProbs) 
foreach(double iflux in iFluxes) {
foreach(double agg in Aggs) {
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
                Job j = new Job(string.Format($"XDGBS-p{p}-{5*n}x{16*n}-agg{agg}-iProb{iProb}-iFlux{iflux}-FphiType{0}-aRI_{true}"), typeof(XESF.XESFMain));// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                j.RetryCount = 2;
                j.MySetCommandLineArguments("--control", $"cs:XESF.XESFHardCodedControl.XDGBS_Cluster(DegE:{p},numX:{5*n},numY:{16*n},agg:{agg},plotInterval:{-1},iProb:{iProb},iflux:{iflux},aRI:true,dbPath:@\"{dbPath}\")");
                j.AdditionalDeploymentFiles.Add(bowShockPoints);
                j.AdditionalDeploymentFiles.Add(LevelSetDB);
                j.Activate(BoSSSshell.ExecutionQueues[0]); 
        };                                                                                                                                                          
    };
    }
};


Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-aRI_True ... 
Deploying executables and additional files ...
Deployment directory: \\dc1\userspace\sebastian\cluster\binaries\XESF_BowShock_ConvStudy2_25082023-XESF2023Aug25_104305.259943
copied 103 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-aRI_True ... 
Deploying executables and additional files ...
Deployment directory: \\dc1\userspace\sebastian\cluster\binaries\XESF_BowShock_ConvStudy2_25082023-XESF2023Aug25_104324.343929
copied 103 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

Deployments so far (0): ;
Success: 0
job submit count: 0


...

Deploying job XDGBS-p3-10x32-agg0.4-iProb0-iFlux1-FphiType0-aRI_True ... 
Deploying executables and additional files ...
Deployment directory: \\dc1\userspace\sebastian\cluster\binaries\XESF_BowShock_ConvStudy2_25082023-XESF2023Aug25_104434.162191
copied 103 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job XDGBS-p3-20x64-agg0.4-iProb0-iFlux1-FphiType0-aRI_True ... 
Deploying executables and additional files ...
Deployment directory: \\dc1\userspace\sebastian\cluster\binaries\XESF_BowShock_ConvStudy2_25082023-XESF2023Aug25_104456.156397
copied 103 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job XDGBS-p3-40x128-agg0.4-iProb0-iFlux1-FphiType0-aRI_True ... 
Deploying executables and additional files ...
Deployment directory: \\dc1\userspace\sebastian\cluster\binaries\XESF_BowShock_ConvStudy2_25082023-XESF2023Aug25_104513.065925
copied 103 files.
   written file: BowShockPoints.txt
   written file: bosss_db_levelSets.zip
   copied 'win\amd64' runtime.
deployment finished.

